In [ ]:
%%bash
. activate idp3
# make clean
make

In [ ]:
%load_ext Cython

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from tail.timestream.boundary_distance import boundary_distance

In [ ]:
from tests.timestream.test_boundary_distance import test_boundary_distance, circular_mask

# Test

In [ ]:
test_boundary_distance()

# Time

In [ ]:
time = []
N = range(7, 11, 1)
for n in N:
    d = 2**n
    r = 2**n // 2 - 1
    print(d, r)
    mask = circular_mask(d, r)
    temp = %timeit -o distance = boundary_distance(mask)
    time.append(temp.best)

In [ ]:
plt.plot(N, time)

In [ ]:
distance = boundary_distance(mask)

In [ ]:

plt.plot(distance[n // 2])

In [ ]:
mask = circular_mask(1024, 128)

In [ ]:
%timeit -o distance = boundary_distance(mask, num_threads=1)